First we'll create an example helper type.

In [46]:
class BitEncoded:
    """Annotation Helper that tracks a bitwidth for an integer."""

    def __init__(self, bitwidth: int):
        self.bitwidth = bitwidth

    def encode(self, value: int) -> str:
        """Encodes the value as a signed integer of self.bitwidth size, returning the binary representation as a string without the `0b` prefix"""
        mask = 2**self.bitwidth - 1

        lowbits = value & mask

        return f"{lowbits:b}"

    def __repr__(self):
        return f"BitEncoded({self.bitwidth})"


# An example
BitEncoded(bitwidth=3).encode(5)


'101'

Next, we'll build a dataclass that uses the helper type.

In [47]:
import dataclasses
import typing


@dataclasses.dataclass
class ExampleDataclass:
    quantity: typing.Annotated[int, BitEncoded(3)] = 0
    price: typing.Annotated[int, BitEncoded(20)] = 0


Given the use of `typing.Annotated`, we can add metadata beyond the type that type-checker are concerned about.

We can access this metadata using `typing.get_type_hints`. By default, this only gives the type-checker metadata, but we can pass a flag to trigger the rest.

In [48]:
typing.get_type_hints(ExampleDataclass())


{'quantity': int, 'price': int}

In [49]:
typing.get_type_hints(ExampleDataclass(), include_extras=True)


{'quantity': typing.Annotated[int, BitEncoded(3)],
 'price': typing.Annotated[int, BitEncoded(20)]}

Now, we can use this metadata at runtime.

In [50]:
data = ExampleDataclass(quantity=3, price=350)
annotation_map = typing.get_type_hints(data, include_extras=True)
typing.get_args(annotation_map["price"])[1]


BitEncoded(20)

In [51]:
data = ExampleDataclass(quantity=3, price=350)
encoders = {
    field: item
    for field, annotation in typing.get_type_hints(data, include_extras=True).items()
    for item in list(typing.get_args(annotation))
    if isinstance(item, BitEncoded)
}
encoders


{'quantity': BitEncoded(3), 'price': BitEncoded(20)}

With this in mind, we can easily define a concept like BitEncoded Dataclass

In [52]:
@dataclasses.dataclass
class BitEncodedExample:
    quantity: typing.Annotated[int, BitEncoded(3)]
    price: typing.Annotated[int, BitEncoded(20)]

    def encode(self) -> str:
        """Encodes this instance into a binary string."""
        output = ""
        # Get the dataclass fields by name and value.
        field_dict = {
            field.name: getattr(self, field.name) for field in dataclasses.fields(self)
        }

        # Get the encoders by name of field and encoder
        encoders = {
            field: item
            for field, annotation in typing.get_type_hints(
                data, include_extras=True
            ).items()
            for item in list(typing.get_args(annotation))
            if isinstance(item, BitEncoded) and field in field_dict.keys()
        }

        for name, encoder in encoders.items():
            value = field_dict[name]
            output += encoder.encode(value)

        return output

# Example
data = BitEncodedExample(quantity=3, price=350)
data.encode()

'11101011110'